In [2]:
!python -m pip install langchain langchain-community langchainhub
!pip install langchain-google-genai
!pip install langchain-google-genai
!pip install chromadb==0.4.22
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI

from langchain.chains.question_answering import load_qa_chain

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 46.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.4/62.4 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.3/41.3 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 69.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 80.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.9/59.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.0/107.0 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 7.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 2

In [3]:
import os
chat_model = genai.configure(api_key="GOOGLE_API_KEY")
os.environ["GOOGLE_API_KEY"] = "GOOGLE_API_KEY"
chat_model = genai.configure(api_key="GOOGLE_API_KEY")
model = ChatGoogleGenerativeAI(model="gemini-pro", convert_system_message_to_human=True)

In [9]:
from langchain.prompts import ChatPromptTemplate
disease_template_str = """ Your job is to predict disease for the given symptoms, But dont make up any information that's not from the
context. If you dont know the answer, say you dont know.
{context}

{question}
"""

disease_template = ChatPromptTemplate.from_template(disease_template_str)
context = "I have fever and yellowish skin"
question = "What are the disease matches with this symptoms"
disease_template.format(context = context,question = question)

from langchain.prompts import (
    PromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate
)

disease_template_str = """Your job is to predict disease for the given symptoms, But dont make up any information that's not from the
context. If you dont know the answer, say you dont know.
{context}
"""

disease_system_prompt = SystemMessagePromptTemplate(
                      prompt = PromptTemplate(
                          input_variables=["context"],template=disease_template_str
                      )
                      )
disease_human_prompt = HumanMessagePromptTemplate(
                      prompt = PromptTemplate(
                          input_variables=["question"],template="{question}"

                      )
                      )
messages = [disease_system_prompt,disease_human_prompt]
disease_prompt_template = ChatPromptTemplate(
                        input_variables=["context","question"],
                        messages = messages
)



chat_model =  ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, convert_system_message_to_human=True) # set the convert_system_message_to_human to true
disease_chain = disease_prompt_template | chat_model

from langchain.prompts import (
    PromptTemplate, SystemMessagePromptTemplate,HumanMessagePromptTemplate,ChatPromptTemplate
)
disease_template_str = """ Your job is to predict disease for the given symptoms, But dont make up any information that's not from the
context. If you dont know the answer, say you dont know.
{context}
"""
disease_system_prompt = SystemMessagePromptTemplate(
                      prompt = PromptTemplate(
                          input_variables=["context"],template=disease_template_str
                      )
                      )
disease_human_prompt = HumanMessagePromptTemplate(
                      prompt = PromptTemplate(
                          input_variables=["question"],template="{question}"

                      )
                      )
messages = [disease_system_prompt,disease_human_prompt]
disease_prompt_template = ChatPromptTemplate(
                        input_variables=["context","question"],
                        messages = messages
)
disease_prompt_template.format_messages(context = context,question = question)


chat_model =  ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3, convert_system_message_to_human=True)
disease_chain = disease_prompt_template | chat_model




In [4]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings

hf_embeddings = HuggingFaceInferenceAPIEmbeddings(
    api_key="HF_API_KEY",
    model_name="sentence-transformers/all-MiniLM-l6-v2"
)


In [16]:
from langchain.document_loaders.csv_loader import CSVLoader
from langchain_community.vectorstores import Chroma

DISEASE_CSV_PATH = "/content/Processed_dataset.csv"
DISEASE_CHROMA_PATH = "/chroma_data"

loader =  CSVLoader(file_path =DISEASE_CSV_PATH)
disease = loader.load()


disease_vector_db= Chroma.from_documents(
    disease,hf_embeddings,persist_directory=DISEASE_CHROMA_PATH
)

In [27]:

from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain.schema.runnable import RunnablePassthrough

DISEASE_CHROMA_PATH = "/chroma_data"

# ...

chat_model =  ChatGoogleGenerativeAI(model="gemini-pro",
                             temperature=0.3,convert_system_message_to_human=True)
disease_vector_db = Chroma(
    persist_directory=DISEASE_CHROMA_PATH,
    embedding_function=hf_embeddings
)

disease_retriever  = disease_vector_db.as_retriever(k=2)

disease_chain = (
    {"context": disease_retriever, "question": RunnablePassthrough()}
    | disease_prompt_template
    | chat_model
    | StrOutputParser()
)

In [15]:
question = """I have been experiencing  chills, continuous_sneezing, watering_from_eyes,Allergy. what is the disease?"""
res = disease_chain.invoke(question)
res

/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


'Allergy'

In [36]:
sym = input("Enter your symptoms: ")
res = disease_chain.invoke(sym)
print("probable disease: ",res)


Enter your symptoms: fever, itching, vomiting


/usr/local/lib/python3.10/dist-packages/langchain_google_genai/chat_models.py:352: UserWarning: Convert_system_message_to_human will be deprecated!
  warnings.warn("Convert_system_message_to_human will be deprecated!")


probable disease:  Jaundice


Malaria
